In [1]:
# -*- coding: utf-8 -*-
"""
Created on May 09 2018

@author: Levani Zandarashvili
modified from a script by acoop_000

Modified from initial scrapper to also parse data and create organ specific files for simpler reading.

Operating system:
The code was written on Python 3 in Jupyter notebook through Anaconda.
Might need correction in specifying folder paths if used in Linux or Mac-os

Description:
This code uses selenium to scrape data from the following url:
https://optn.transplant.hrsa.gov/data/view-data-reports/build-advanced/

In short, program will open the url, and go through every center and extract the data, which will be written
into individual csv files.
The data is in a form of a table, but it is a bit complex looking at first. Data contains following columns:
"waiting time", "Score", "ABO", "(columns for individual organs, as well as all the organs together)"

Note that different centers will have different sets of organs.

If the center contains no data, then no file will be created for it.

Inputs:
The only necessary input for the code is url, which is already specified. It is possible to change the set
of parameters, but this needs to be done with care (to avoid errors). Note that no more than three different
parameters can be specified (limitation of the original web page).

Dependencies:
selenium, datetime, os, re, pandas
geckodriver must be in PATH
firefox.exe must be in PATH

Output:
'Scrapper_Data' directory
Contains .csv file for each transplant center
filename corresponds ot the name of the center
File format: 

Scraper still crashes sometimes (PAVA-TX1 VA Pittsburgh Healthcare System)

"""

#!/usr/bin/env python

import os, datetime

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import UnexpectedTagNameException
#import pandas as pd
import re


In [2]:
# specify url and current time
url = 'https://optn.transplant.hrsa.gov/data/view-data-reports/build-advanced/'
today = str(datetime.datetime.now()).split()[0]

In [3]:
# this function will open url, choose a waiting list category and iterate through every available center.
# for each center, it will call a function "crawl_center_test" and extract the information available at that center.
def iterate_centers():
    # specify browser and url
    browser = webdriver.Firefox()
    browser.get(url)
    
    # choose a data category: waiting list
    select_category = Select(browser.find_element_by_id("category"))
    select_category.select_by_index(3)
    
    select_center = Select(browser.find_element_by_id("slice6"))
    
    for center_i in range(1, len(select_center.options)):
        select_center.select_by_index(center_i)
        selected = select_center.first_selected_option
        center = selected.text
        print("center: ", center)
        crawl_center_test(center_i)
        print('scraped')
        
    browser.close()

In [4]:
# number of patients higher than 999 in the original data is written using comma (i.e. 1,255)
# this function removes commas from such numbers
def remove_commas(num_patients):
    new_num = []
    for num in num_patients:
        new_num = new_num + [num.replace(',','')]
    return(new_num)

In [5]:
# this function will scrape data for individual centers. Center will be scpecified by iterate_centers function.
def crawl_center_test(center_i):
    
    # specify browser and url
    browser = webdriver.Firefox()
    browser.get(url)
    
    # creating directory to save data (if not already created)
    if not os.path.isdir('./Scrapper_Data'):
        os.mkdir('./Scrapper_Data')
    
    # creating sub-directory for todays data (if not already created)
    if not os.path.isdir('./Scrapper_Data/%s' % (today)):
        os.mkdir('./Scrapper_Data/%s' % (today))
    
    # select options
    try:
        # choose a data category: waiting list
        # print('select category')
        select_category = Select(browser.find_element_by_id("category"))
        select_category.select_by_index(3)
    
        # Choose report columns: organs
        # print('select organs')
        select_columns = Select(browser.find_element_by_id("col1"))
        select_columns.select_by_index(15)
    
        # Choose report rows: 'waiting time', 'waiting list status (score)', 'abo'
        # print('select waiting time')
        select_rows1 = Select(browser.find_element_by_id("row1"))
        select_rows1.select_by_index(25) # waiting time
        
        # print('select waiting list status')
        select_rows2 = Select(browser.find_element_by_id("row2"))
        select_rows2.select_by_index(18) # age
    
        # print('select abo')
        select_rows3 = Select(browser.find_element_by_id("row3"))
        select_rows3.select_by_index(1) # blood type
    
        # select center (given as input to the function)
        # print('select center')
        select_center = Select(browser.find_element_by_id("slice6"))
        select_center.select_by_index(center_i)
        
        # get center name as "center"
        selected = select_center.first_selected_option
        center = selected.text
        
        # click "go" on the page after specifying all the options
        browser.find_element_by_id('DataAdvancedSubmit').click()
        
        # following section will extract data from the tables on the page
        tables = browser.find_elements_by_css_selector('.dataGrid')
        
        rows = []
        columns = []
        for i in range(len(tables)):
            # magic happens here
            table = browser.find_element_by_xpath('//*[@id="DataReportContainer"]/table[%d]' % (3+2*i)).text
            organs = table.split('\n')[0]
            organs = organs.replace(' / ', '/')
            organs = organs.replace('All Organs', 'All_Organs')
            organs = organs.split(' ')[8:]
            #print('organs')
            #print(organs)
            
            n_organs = len(organs) # number of organs for this center
            
            # create header (list of columns) for this center 
            if not columns:
                columns = ['time', 'score', 'ABO']
                columns = columns + organs
                rows.append(','.join(columns)) # add header
            
            for line in table.split('\n'):
                if ((not 'All Organs' in line) & (not 'All_Organs' in line)): # skip header line for each table
                    line_split = list(filter(None, re.split(r'\s{2,}', line)))   # split line with double space to generate groups
                    if len(line_split) == 4: # line includes time, score, blood type
                        wait_time, score, abo = line_split[0:3]
                        patient_num = remove_commas(line_split[3].split(' '))
                        line_combined = [wait_time, score, abo] + patient_num
                        line_combined = ",".join(line_combined)
                        rows.append(line_combined)
                    elif len(line_split) == 3: # line includes score, blood type
                        score, abo = line_split[0:2]
                        patient_num = remove_commas(line_split[2].split(' '))
                        line_combined = [wait_time, score, abo] + patient_num
                        line_combined = ",".join(line_combined)
                        rows.append(line_combined)
                    elif len(line_split) == 2: # line includes blood type
                        abo = line_split[0]
                        patient_num = remove_commas(line_split[1].split(' '))
                        line_combined = [wait_time, score, abo] + patient_num
                        line_combined = ",".join(line_combined)
                        rows.append(line_combined)

    
        # Write to csv file
        
        if rows:
            file_name = "%s.csv" % (center.replace("/", "-"))
        
            write_file('Scrapper_Data/%s/%s' % (today, file_name), rows)        
        
        # I noticed that problem happens due to the web page not opening properly
        # So if the error is encountered, I will crawl the center again
        # even if the center has no data, it shouldn't give an error
    except UnexpectedTagNameException:
        print("encountered problem with %s, retrying" % (center_i))
        crawl_center_test(center_i)
    except NoSuchElementException:
        print("encountered problem with %s, retrying" % (center_i))
        crawl_center_test(center_i)
    
    browser.close()

In [6]:
# function to write data to file
def write_file(name, data):
    with open(name, 'w') as f:
        for row in data:
            f.write(row + '\n')

In [7]:
import time

start_time = time.time()
iterate_centers()
elapsed_time = time.time() - start_time
print(elapsed_time)

center:  ALAM-TX1 Univ of South Alabama Med Ctr
scraped
center:  ALCH-TX1 Children's of Alabama
organs
['All_Organs', 'Kidney', 'Liver', 'Heart']
['time', 'score', 'ABO', 'All_Organs', 'Kidney', 'Liver', 'Heart']
 All Time,All Types,All ABO,38,28,4,6
 All Time,All Types,O,21,15,1,5
 All Time,All Types,A,10,8,2,0
 All Time,All Types,B,7,5,1,1
 All Time,Liver MELD / PELD <15,All ABO,2,0,2,0
 All Time,Liver MELD / PELD <15,O,1,0,1,0
 All Time,Liver MELD / PELD <15,A,1,0,1,0
 All Time,Liver MELD / PELD 15-19,All ABO,1,0,1,0
 All Time,Liver MELD / PELD 15-19,A,1,0,1,0
 All Time,Liver MELD / PELD 35+,All ABO,1,0,1,0
 All Time,Liver MELD / PELD 35+,B,1,0,1,0
 All Time,Heart Status 1A,All ABO,3,0,0,3
 All Time,Heart Status 1A,O,2,0,0,2
 All Time,Heart Status 1A,B,1,0,0,1
 All Time,Heart Status 1B,All ABO,1,0,0,1
 All Time,Heart Status 1B,O,1,0,0,1
 All Time,Heart Status 7 (Inactive),All ABO,2,0,0,2
 All Time,Heart Status 7 (Inactive),O,2,0,0,2
 All Time,Kidney/Pancreas/Kidney-Pancreas: Active,

organs
['All_Organs', 'Kidney', 'Liver', 'Pancreas', 'Kidney/Pancreas', 'Heart', 'Lung', 'Heart/Lung', 'Intestine']
['time', 'score', 'ABO', 'All_Organs', 'Kidney', 'Liver', 'Pancreas', 'Kidney/Pancreas', 'Heart', 'Lung', 'Heart/Lung', 'Intestine']
 All Time,Lung/Heart-Lung: Lung Allocation Score 30 to < 35,O,3,0,0,0,0,0,2,1,0
 All Time,Lung/Heart-Lung: Lung Allocation Score 30 to < 35,A,2,0,0,0,0,0,2,0,0
 All Time,Lung/Heart-Lung: Lung Allocation Score 35 to < 40,All ABO,8,0,0,0,0,0,8,0,0
 All Time,Lung/Heart-Lung: Lung Allocation Score 35 to < 40,O,2,0,0,0,0,0,2,0,0
 All Time,Lung/Heart-Lung: Lung Allocation Score 35 to < 40,A,3,0,0,0,0,0,3,0,0
 All Time,Lung/Heart-Lung: Lung Allocation Score 35 to < 40,B,3,0,0,0,0,0,3,0,0
 All Time,Lung/Heart-Lung: Lung Allocation Score 40 to 50,All ABO,4,0,0,0,0,0,4,0,0
 All Time,Lung/Heart-Lung: Lung Allocation Score 40 to 50,O,2,0,0,0,0,0,2,0,0
 All Time,Lung/Heart-Lung: Lung Allocation Score 40 to 50,A,2,0,0,0,0,0,2,0,0
 All Time,Lung/Heart-Lung

organs
['All_Organs', 'Kidney', 'Liver', 'Pancreas', 'Kidney/Pancreas', 'Heart', 'Lung', 'Heart/Lung', 'Intestine']
['time', 'score', 'ABO', 'All_Organs', 'Kidney', 'Liver', 'Pancreas', 'Kidney/Pancreas', 'Heart', 'Lung', 'Heart/Lung', 'Intestine']
 90 Days to < 6 Months,Lung/Heart-Lung: Lung Allocation Score 20 to < 30,AB,2,0,0,0,0,0,1,1,0
 90 Days to < 6 Months,Lung/Heart-Lung: Lung Allocation Score >=50,All ABO,1,0,0,0,0,0,1,0,0
 90 Days to < 6 Months,Lung/Heart-Lung: Lung Allocation Score >=50,A,1,0,0,0,0,0,1,0,0
 90 Days to < 6 Months,Kidney/Pancreas/Kidney-Pancreas: Active,All ABO,63,61,0,0,2,0,0,0,0
 90 Days to < 6 Months,Kidney/Pancreas/Kidney-Pancreas: Active,O,34,33,0,0,1,0,0,0,0
 90 Days to < 6 Months,Kidney/Pancreas/Kidney-Pancreas: Active,A,17,16,0,0,1,0,0,0,0
 90 Days to < 6 Months,Kidney/Pancreas/Kidney-Pancreas: Active,B,11,11,0,0,0,0,0,0,0
 90 Days to < 6 Months,Kidney/Pancreas/Kidney-Pancreas: Active,AB,1,1,0,0,0,0,0,0,0
 90 Days to < 6 Months,Kidney/Panc/Kidney-Panc:

organs
['All_Organs', 'Kidney', 'Liver', 'Pancreas', 'Kidney/Pancreas', 'Heart', 'Lung', 'Heart/Lung', 'Intestine']
['time', 'score', 'ABO', 'All_Organs', 'Kidney', 'Liver', 'Pancreas', 'Kidney/Pancreas', 'Heart', 'Lung', 'Heart/Lung', 'Intestine']
 2 Years to < 3 Years,Kidney/Panc/Kidney-Panc: Temporarily inactive,All ABO,18,17,0,0,1,0,0,0,0
 2 Years to < 3 Years,Kidney/Panc/Kidney-Panc: Temporarily inactive,O,10,10,0,0,0,0,0,0,0
 2 Years to < 3 Years,Kidney/Panc/Kidney-Panc: Temporarily inactive,A,5,5,0,0,0,0,0,0,0
 2 Years to < 3 Years,Kidney/Panc/Kidney-Panc: Temporarily inactive,B,1,0,0,0,1,0,0,0,0
 2 Years to < 3 Years,Kidney/Panc/Kidney-Panc: Temporarily inactive,AB,2,2,0,0,0,0,0,0,0
 3 Years to < 5 Years,All Types,All ABO,443,430,7,1,1,2,2,0,0
 3 Years to < 5 Years,All Types,O,251,242,4,1,1,2,1,0,0
 3 Years to < 5 Years,All Types,A,110,107,2,0,0,0,1,0,0
 3 Years to < 5 Years,All Types,B,79,78,1,0,0,0,0,0,0
 3 Years to < 5 Years,All Types,AB,3,3,0,0,0,0,0,0,0
 3 Years to < 5 Yea

scraped
center:  ARBH-TX1 Baptist Medical Center
organs
['All_Organs', 'Heart']
['time', 'score', 'ABO', 'All_Organs', 'Heart']
 All Time,All Types,All ABO,37,37
 All Time,All Types,O,24,24
 All Time,All Types,A,8,8
 All Time,All Types,B,5,5
 All Time,Heart Status 1A,All ABO,5,5
 All Time,Heart Status 1A,O,5,5
 All Time,Heart Status 1B,All ABO,18,18
 All Time,Heart Status 1B,O,8,8
 All Time,Heart Status 1B,A,7,7
 All Time,Heart Status 1B,B,3,3
 All Time,Heart Status 2,All ABO,2,2
 All Time,Heart Status 2,O,1,1
 All Time,Heart Status 2,B,1,1
 All Time,Heart Status 7 (Inactive),All ABO,12,12
 All Time,Heart Status 7 (Inactive),O,10,10
 All Time,Heart Status 7 (Inactive),A,1,1
 All Time,Heart Status 7 (Inactive),B,1,1
 < 30 Days,All Types,All ABO,1,1
 < 30 Days,All Types,B,1,1
 < 30 Days,Heart Status 1B,All ABO,1,1
 < 30 Days,Heart Status 1B,B,1,1
 30 to < 90 Days,All Types,All ABO,6,6
 30 to < 90 Days,All Types,O,4,4
 30 to < 90 Days,All Types,A,2,2
 30 to < 90 Days,Heart Status 1B,All A

...

organs
['All_Organs', 'Kidney', 'Liver', 'Kidney/Pancreas', 'Heart']
['time', 'score', 'ABO', 'All_Organs', 'Kidney', 'Liver', 'Kidney/Pancreas', 'Heart']
 1 Year to < 2 Years,All Types,A,26,18,7,0,1
 1 Year to < 2 Years,All Types,B,14,11,2,0,1
 1 Year to < 2 Years,All Types,AB,1,0,1,0,0
 1 Year to < 2 Years,Liver Status 7 (Inactive),All ABO,3,0,3,0,0
 1 Year to < 2 Years,Liver Status 7 (Inactive),O,2,0,2,0,0
 1 Year to < 2 Years,Liver Status 7 (Inactive),A,1,0,1,0,0
 1 Year to < 2 Years,Liver MELD / PELD <15,All ABO,8,0,8,0,0
 1 Year to < 2 Years,Liver MELD / PELD <15,O,2,0,2,0,0
 1 Year to < 2 Years,Liver MELD / PELD <15,A,3,0,3,0,0
 1 Year to < 2 Years,Liver MELD / PELD <15,B,2,0,2,0,0
 1 Year to < 2 Years,Liver MELD / PELD <15,AB,1,0,1,0,0
 1 Year to < 2 Years,Liver MELD / PELD 15-19,All ABO,2,0,2,0,0
 1 Year to < 2 Years,Liver MELD / PELD 15-19,O,2,0,2,0,0
 1 Year to < 2 Years,Liver MELD / PELD 20-24,All ABO,3,0,3,0,0
 1 Year to < 2 Years,Liver MELD / PELD 20-24,O,1,0,1,0,0
 1 Yea

organs
['All_Organs', 'Kidney', 'Liver', 'Pancreas', 'Kidney/Pancreas', 'Heart', 'Lung']
['time', 'score', 'ABO', 'All_Organs', 'Kidney', 'Liver', 'Pancreas', 'Kidney/Pancreas', 'Heart', 'Lung']
 All Time,Heart Status 7 (Inactive),B,1,0,0,0,0,1,0
 All Time,Lung/Heart-Lung: Lung Allocation Score 30 to < 35,All ABO,7,0,0,0,0,0,7
 All Time,Lung/Heart-Lung: Lung Allocation Score 30 to < 35,O,5,0,0,0,0,0,5
 All Time,Lung/Heart-Lung: Lung Allocation Score 30 to < 35,A,1,0,0,0,0,0,1
 All Time,Lung/Heart-Lung: Lung Allocation Score 30 to < 35,B,1,0,0,0,0,0,1
 All Time,Lung/Heart-Lung: Lung Allocation Score 35 to < 40,All ABO,5,0,0,0,0,0,5
 All Time,Lung/Heart-Lung: Lung Allocation Score 35 to < 40,O,4,0,0,0,0,0,4
 All Time,Lung/Heart-Lung: Lung Allocation Score 35 to < 40,A,1,0,0,0,0,0,1
 All Time,Lung/Heart-Lung: Lung Allocation Score 40 to 50,All ABO,1,0,0,0,0,0,1
 All Time,Lung/Heart-Lung: Lung Allocation Score 40 to 50,O,1,0,0,0,0,0,1
 All Time,Lung/Heart-Lung: Inactive,All ABO,7,0,0,0,0,0

organs
['All_Organs', 'Kidney', 'Liver', 'Pancreas', 'Kidney/Pancreas', 'Heart', 'Lung']
['time', 'score', 'ABO', 'All_Organs', 'Kidney', 'Liver', 'Pancreas', 'Kidney/Pancreas', 'Heart', 'Lung']
 90 Days to < 6 Months,Kidney/Panc/Kidney-Panc: Temporarily inactive,O,14,13,0,0,1,0,0
 90 Days to < 6 Months,Kidney/Panc/Kidney-Panc: Temporarily inactive,A,7,7,0,0,0,0,0
 90 Days to < 6 Months,Kidney/Panc/Kidney-Panc: Temporarily inactive,B,2,2,0,0,0,0,0
 90 Days to < 6 Months,Kidney/Panc/Kidney-Panc: Temporarily inactive,AB,1,1,0,0,0,0,0
 6 Months to < 1 Year,All Types,All ABO,182,155,19,0,1,2,5
 6 Months to < 1 Year,All Types,O,77,62,8,0,1,2,4
 6 Months to < 1 Year,All Types,A,64,58,6,0,0,0,0
 6 Months to < 1 Year,All Types,B,34,29,4,0,0,0,1
 6 Months to < 1 Year,All Types,AB,7,6,1,0,0,0,0
 6 Months to < 1 Year,Liver Status 7 (Inactive),All ABO,3,0,3,0,0,0,0
 6 Months to < 1 Year,Liver Status 7 (Inactive),O,3,0,3,0,0,0,0
 6 Months to < 1 Year,Liver MELD / PELD <15,All ABO,6,0,6,0,0,0,0
 6 M

organs
['All_Organs', 'Kidney', 'Liver', 'Pancreas', 'Kidney/Pancreas', 'Heart', 'Lung']
['time', 'score', 'ABO', 'All_Organs', 'Kidney', 'Liver', 'Pancreas', 'Kidney/Pancreas', 'Heart', 'Lung']
 3 Years to < 5 Years,Liver Status 7 (Inactive),A,2,0,2,0,0,0,0
 3 Years to < 5 Years,Liver MELD / PELD <15,All ABO,3,0,3,0,0,0,0
 3 Years to < 5 Years,Liver MELD / PELD <15,A,1,0,1,0,0,0,0
 3 Years to < 5 Years,Liver MELD / PELD <15,B,1,0,1,0,0,0,0
 3 Years to < 5 Years,Liver MELD / PELD <15,AB,1,0,1,0,0,0,0
 3 Years to < 5 Years,Liver MELD / PELD 15-19,All ABO,2,0,2,0,0,0,0
 3 Years to < 5 Years,Liver MELD / PELD 15-19,O,1,0,1,0,0,0,0
 3 Years to < 5 Years,Liver MELD / PELD 15-19,A,1,0,1,0,0,0,0
 3 Years to < 5 Years,Liver MELD / PELD 25-29,All ABO,2,0,2,0,0,0,0
 3 Years to < 5 Years,Liver MELD / PELD 25-29,O,1,0,1,0,0,0,0
 3 Years to < 5 Years,Liver MELD / PELD 25-29,A,1,0,1,0,0,0,0
 3 Years to < 5 Years,Heart Status 2,All ABO,2,0,0,0,0,2,0
 3 Years to < 5 Years,Heart Status 2,O,1,0,0,0,0,1,

scraped
center:  WVWU-TX1 WV University Hospital
scraped
center:  Unknown Transplant Center
scraped
5796.5013422966
